In [0]:
!pip install kaggle

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "Suman135@"
os.environ["KAGGLE_KEY"] = "KGAT_e47bd20d08aee82855a907087a6362de"

print("Kaggle credentials configured!")

In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh


In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh


In [0]:
%restart_python

### Creating a Parametarized Function for loading data


In [0]:
def load_ecommerce_data(month):
    """
    Load ecommerce CSV data for a given month (e.g. 'Oct', 'Nov')
    """
    path = f"/Volumes/workspace/ecommerce/ecommerce_data/2019-{month}.csv"
    
    return (
        spark.read
             .option("header", "true")
             .option("inferSchema", "true")
             .csv(path)
    )

### Initial Inspection --- Rows count & DataFrame View

In [0]:
# Load your data
events = load_ecommerce_data("Oct")

# Verify it's working
print(f"✅ Ready to go! Loaded {events.count():,} events")
display(events)



In [0]:
display(events)

### Day 1

In [0]:
# 1️⃣ Create sample sales datadata
data = [
    Row(order_id=1, product="Laptop", category="Electronics", price=75000, quantity=1),
    Row(order_id=2, product="Headphones", category="Electronics", price=3000, quantity=2),
    Row(order_id=3, product="Chair", category="Furniture", price=4500, quantity=1),
    Row(order_id=4, product="Desk", category="Furniture", price=12000, quantity=1),
    Row(order_id=5, product="Monitor", category="Electronics", price=18000, quantity=2),
    Row(order_id=6, product="Keyboard", category="Electronics", price=2500, quantity=3)
]

sales_df = spark.createDataFrame(data)

# 2️⃣ View data & schema
from pyspark.sql.functions import col, sum, avg, count
display(sales_df)

sales_df.printSchema()

# 3️⃣ Add total amount column
sales_enriched = sales_df.withColumn(
    "total_amount",
    col("price") * col("quantity")
)

display(sales_enriched)

# 4️⃣ Category-level sales summary
category_summary = (
    sales_enriched
    .groupBy("category")
    .agg(
        sum("total_amount").alias("total_revenue"),
        avg("price").alias("avg_price"),
        count("order_id").alias("total_orders")
    )
    .orderBy(col("total_revenue").desc())
)

display(category_summary)

print("✅ Day 1 PySpark DataFrame operations completed successfully!")


### Day 2

In [0]:
# Load data
events = spark.read.csv("/Volumes/workspace/challenge_days/day2/sample_ecommerce_day2.csv", header=True, inferSchema=True)

# Basic operations
events.select("event_type", "product_name", "price").show(10)
events.filter("price > 100").count()
events.groupBy("event_type").count().show()
top_brands = events.groupBy("brand").count().orderBy("count", ascending=False).limit(5)
display(top_brands)

### Day 3

In [0]:
# Day 3
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Top 5 products by revenue
revenue = events.filter(F.col("event_type") == "purchase") \
    .groupBy("product_id") \
    .agg(F.sum("price").alias("revenue")) \
    .orderBy(F.desc("revenue")).limit(5)

# Running total per user
window = Window.partitionBy("user_id").orderBy("event_time")
events.withColumn("cumulative_events", F.count("*").over(window))

# Conversion rate by category
conv_rate =events.groupBy("category_code", "event_type").count() \
    .groupBy("category_code").pivot("event_type").sum("count") \
    .withColumn("conversion_rate", F.col("purchase")/F.col("view")*100)
display(revenue)
display(conv_rate.limit(5))

### Day 4

In [0]:
# Convert to Delta
events.write.format("delta").mode("overwrite").save("/Volumes/workspace/ecommerce/ecommerce_data/delta")

# Create managed table
events.write.mode("overwrite").format("delta").saveAsTable("events_table")

# SQL approach


# spark.sql("""
#     CREATE TABLE events_delta
#     USING DELTA
#     AS SELECT * FROM events_table
# """)

# Test schema enforcement
try:
    wrong_schema = spark.createDataFrame([("a","b","c")], ["x","y","z"])
    wrong_schema.write.format("delta").mode("append").save("/delta/events")
except Exception as e:
    print(f"Schema enforcement: {e}")


In [0]:
# Check the table exists & basic info (SQL)

spark.sql("SHOW TABLES").show()


In [0]:
# Verify table format = DELTA

display(
    spark.sql("DESCRIBE DETAIL events_table")
)


### Day 5 -Delta Lake Advanced

**🔹 STEP 1: Importing Libraries**

In [0]:
from delta.tables import DeltaTable
from pyspark.sql import Row
from pyspark.sql.functions import to_timestamp

**🔹STEP 2: Load Delta Table**

In [0]:
DELTA_PATH = "/Volumes/workspace/ecommerce/ecommerce_data/delta"
deltaTable = DeltaTable.forPath(spark, DELTA_PATH)

**🔹STEP 3: View Data BEFORE UPSERT**

In [0]:
print("📌 BEFORE UPSERT (Current Delta Table)")
display(
    deltaTable.toDF()
    .orderBy("event_time", "product_id")
)

**🔹 STEP 4: Create SOURCE Data for UPSERT (Incremental Data)**

In [0]:
updates_data = [

    Row("2024-01-01 10:05:00", "purchase", 101, "Laptop",
        "Electronics", "Lenovo", 75000, 1),

    Row("2024-01-01 10:25:00", "view", 106, "Keyboard",
        "Electronics", "Logitech", 2500, 4),
    
    Row("2024-01-01 10:18:00", "purchase", 105, "Monitor",
        "Electronics", "Samsung", 17500, 1),

    Row("2024-01-01 11:00:00", "view", 109, "Tablet",
        "Electronics", "Apple", 55000, 1),
    
    Row("2024-01-01 11:05:00", "purchase", 110, "Smartwatch",
        "Electronics", "Samsung", 22000, 1)
]

updates = spark.createDataFrame(
    updates_data,
    [
        "event_time", "event_type", "product_id", "product_name",
        "category", "brand", "price", "quantity"
    ]
).withColumn("event_time", to_timestamp("event_time"))

print("📌 SOURCE DATA FOR UPSERT")
display(updates)


**🔹 STEP 5: Perform UPSERT using MERGE**

In [0]:
deltaTable.alias("t").merge(
    updates.alias("s"),
    """
    t.event_time = s.event_time
    AND t.product_id = s.product_id
    """
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

**🔹 STEP 6: View Data AFTER UPSERT**

In [0]:
print("📌 AFTER UPSERT (Delta Table Updated)")
display(
    deltaTable.toDF()
    .orderBy("event_time", "product_id")
)

**🔹 STEP 7: View Delta Table History (Time Travel Metadata)**

In [0]:
print("📌 DELTA TABLE HISTORY")
display(deltaTable.history())


**🔹 STEP 8: Time Travel (Query Older Version)**

In [0]:
print("📌 TIME TRAVEL – VERSION 0")
spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .load(DELTA_PATH) \
    .orderBy("event_time", "product_id") \
    .show()

**🔹 STEP 9: Register Table for SQL**

In [0]:
spark.read.format("delta") \
    .load(DELTA_PATH) \
    .createOrReplaceTempView("events_table_tmp")

**STEP 10: OPTIMIZE (Performance Tuning)**

In [0]:
%sql
OPTIMIZE workspace.default.events_table

ZORDER BY (event_type, product_id);

**🔹 STEP 11: VACUUM (Cleanup Old Files)**

In [0]:
spark.sql("VACUUM events_table RETAIN 168 HOURS")